In [2]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Using numpy backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [3]:
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467)
print(Y.shape)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()

(467,)


In [3]:
inputs = []
max_length = 0
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=0)
    a = np.log(a)
    inputs.append(a)

In [4]:
use = np.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation

In [5]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))

In [6]:
b.shape,use.shape

((467, 53, 200), (467, 467))

In [7]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [8]:
t_s = []
t_e = []
t_y = []
for E,S,Y in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape

(torch.Size([141, 467]), torch.Size([141, 53, 200]), torch.Size([141]))

# glove_rnn_max: 0.6453900709219859 24轮

In [42]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN_max(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=64,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,(h_n,c_n)=self.rnn(x.float())
        output_in_last_timestep = self.maxpool(output)
        output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        x=self.out(output_in_last_timestep)
        return x

In [95]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN_max()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN_max(
  (rnn): LSTM(200, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.23404255319148937 loss: 1.900092601776123
2: accuracy:0.36879432624113473 loss: 1.774414300918579
3: accuracy:0.3900709219858156 loss: 1.5358353853225708
4: accuracy:0.425531914893617 loss: 1.3111639022827148
5: accuracy:0.5531914893617021 loss: 1.1252729892730713
6: accuracy:0.574468085106383 loss: 0.908991813659668
7: accuracy:0.6099290780141844 loss: 0.7280575633049011
8: accuracy:0.5957446808510638 loss: 0.4345526695251465
9: accuracy:0.5886524822695035 loss: 0.4153765141963959
10: accuracy:0.5177304964539007 loss: 0.2946902811527252
11: accuracy:0.574468085106383 loss: 0.29680508375167847
12: accuracy:0.6028368794326241 loss: 0.24011000990867615
13: accuracy:0.6099290780141844 loss: 0.20296014845371246
14: accuracy:0.5886524822695035 loss: 0.09348371624946594


135: accuracy:0.6028368794326241 loss: 0.00021398407989181578
136: accuracy:0.6028368794326241 loss: 0.0002589430077932775
137: accuracy:0.6028368794326241 loss: 0.0002392223832430318
138: accuracy:0.6028368794326241 loss: 0.00023429734574165195
139: accuracy:0.6028368794326241 loss: 0.00023956298537086695
140: accuracy:0.6028368794326241 loss: 0.00023491722822654992
141: accuracy:0.6028368794326241 loss: 0.0002507959143258631
142: accuracy:0.6028368794326241 loss: 0.0002563544549047947
143: accuracy:0.6028368794326241 loss: 0.00019389561202842742
144: accuracy:0.6028368794326241 loss: 0.00021977424330543727
145: accuracy:0.6028368794326241 loss: 0.0001906463148770854
146: accuracy:0.6028368794326241 loss: 0.00022589138825424016
147: accuracy:0.6028368794326241 loss: 0.00021901812579017133
148: accuracy:0.6099290780141844 loss: 0.00020920889801345766
149: accuracy:0.6099290780141844 loss: 0.00021446772734634578
150: accuracy:0.6099290780141844 loss: 0.00021012169599998742
151: accuracy

# glove_bi-rnn_max: 0.7021276595744681 11轮

In [9]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class bi_RNN_max(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=True
        )
        self.out=torch.nn.Linear(in_features=128,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,(h_n,c_n)=self.rnn(x.float())
        output_in_last_timestep = self.maxpool(output)
        output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        x=self.out(output_in_last_timestep)
        return x

In [10]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = bi_RNN_max()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

bi_RNN_max(
  (rnn): LSTM(200, 64, batch_first=True, bidirectional=True)
  (out): Linear(in_features=128, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.23404255319148937 loss: 1.870179295539856
2: accuracy:0.5390070921985816 loss: 1.6827722787857056
3: accuracy:0.574468085106383 loss: 1.3002525568008423
4: accuracy:0.5460992907801419 loss: 0.9892982244491577
5: accuracy:0.624113475177305 loss: 0.7408919334411621
6: accuracy:0.5957446808510638 loss: 0.47758299112319946
7: accuracy:0.6382978723404256 loss: 0.3864317238330841
8: accuracy:0.6524822695035462 loss: 0.2614588737487793
9: accuracy:0.6879432624113475 loss: 0.1430911272764206
10: accuracy:0.6595744680851063 loss: 0.11513009667396545
11: accuracy:0.7021276595744681 loss: 0.06006189063191414
12: accuracy:0.7021276595744681 loss: 0.027230773121118546
13: accuracy:0.6808510638297872 loss: 0.019386181607842445
14: accuracy:0.6666666666666

134: accuracy:0.6312056737588653 loss: 9.211131691699848e-05
135: accuracy:0.6312056737588653 loss: 0.00010104179091285914
136: accuracy:0.6312056737588653 loss: 9.264946129405871e-05
137: accuracy:0.6312056737588653 loss: 8.654594421386719e-05
138: accuracy:0.624113475177305 loss: 8.63824607222341e-05
139: accuracy:0.624113475177305 loss: 0.00010326249321224168
140: accuracy:0.624113475177305 loss: 9.602138015907258e-05
141: accuracy:0.624113475177305 loss: 9.869167115539312e-05
142: accuracy:0.624113475177305 loss: 9.86303566605784e-05
143: accuracy:0.624113475177305 loss: 7.38348244340159e-05
144: accuracy:0.624113475177305 loss: 8.80922598298639e-05
145: accuracy:0.624113475177305 loss: 8.296285523101687e-05
146: accuracy:0.624113475177305 loss: 0.00010580335219856352
147: accuracy:0.624113475177305 loss: 7.696151442360133e-05
148: accuracy:0.624113475177305 loss: 7.896423630882055e-05
149: accuracy:0.624113475177305 loss: 6.988389213802293e-05
150: accuracy:0.624113475177305 loss:

# glove:0.44680851063829785

In [12]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN_old(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=64,out_features=7)

    def forward(self,sen,x):
        output,(h_n,c_n)=self.rnn(x.float())
        output_in_last_timestep=h_n[-1,:,:]
        x=self.out(output_in_last_timestep)
        return x

In [ ]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN_old()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,501): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN_old(
  (rnn): LSTM(200, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=7, bias=True)
)
1: accuracy:0.19148936170212766 loss: 1.9497759342193604
2: accuracy:0.19148936170212766 loss: 1.9573512077331543
3: accuracy:0.10638297872340426 loss: 1.9416414499282837
4: accuracy:0.10638297872340426 loss: 1.9456526041030884
5: accuracy:0.10638297872340426 loss: 1.9020291566848755
6: accuracy:0.10638297872340426 loss: 1.95263671875
7: accuracy:0.19148936170212766 loss: 1.9122650623321533
8: accuracy:0.19148936170212766 loss: 1.942103385925293
9: accuracy:0.1773049645390071 loss: 1.905679702758789
10: accuracy:0.19148936170212766 loss: 1.9215768575668335
11: accuracy:0.19148936170212766 loss: 1.9475725889205933
12: accuracy:0.19148936170212766 loss: 1.92251455783844
13: accuracy:0.2198581560283688 loss: 2.0140671730041504
14: accuracy:0.19148936170212766 loss: 1.923090934753418
15: accuracy:0.19148936170212766 loss: 1.9338258504867554
16: accuracy:0.2127659574468085 loss: 1.

142: accuracy:0.3333333333333333 loss: 0.29946187138557434
143: accuracy:0.3120567375886525 loss: 0.08196274936199188
144: accuracy:0.3049645390070922 loss: 0.3095340132713318
145: accuracy:0.3404255319148936 loss: 0.32889673113822937
146: accuracy:0.3262411347517731 loss: 0.1622338891029358
147: accuracy:0.3404255319148936 loss: 0.18762880563735962
148: accuracy:0.3546099290780142 loss: 0.1792835146188736
149: accuracy:0.3404255319148936 loss: 0.3286314904689789
150: accuracy:0.3475177304964539 loss: 0.10441148281097412
151: accuracy:0.36879432624113473 loss: 0.19653062522411346
152: accuracy:0.3617021276595745 loss: 0.149704247713089
153: accuracy:0.3617021276595745 loss: 0.11561275273561478
154: accuracy:0.3546099290780142 loss: 0.18378540873527527
155: accuracy:0.3475177304964539 loss: 0.16312937438488007
156: accuracy:0.3475177304964539 loss: 0.2698037922382355
157: accuracy:0.3404255319148936 loss: 0.16241727769374847
158: accuracy:0.3333333333333333 loss: 0.2321598380804062
159:

282: accuracy:0.3262411347517731 loss: 0.019658265635371208
283: accuracy:0.3191489361702128 loss: 0.06760052591562271
284: accuracy:0.3262411347517731 loss: 0.07734581083059311
285: accuracy:0.3262411347517731 loss: 0.05724101886153221
286: accuracy:0.3333333333333333 loss: 0.08135704696178436
287: accuracy:0.3333333333333333 loss: 0.06198201701045036
288: accuracy:0.3475177304964539 loss: 0.040318407118320465
289: accuracy:0.3333333333333333 loss: 0.08047070354223251
290: accuracy:0.3333333333333333 loss: 0.044031694531440735
291: accuracy:0.3120567375886525 loss: 0.061553213745355606
292: accuracy:0.3120567375886525 loss: 0.024109888821840286
293: accuracy:0.3120567375886525 loss: 0.03153865411877632
294: accuracy:0.3191489361702128 loss: 0.00930163823068142
295: accuracy:0.3262411347517731 loss: 0.01284746453166008
296: accuracy:0.3404255319148936 loss: 0.04430127143859863
297: accuracy:0.3404255319148936 loss: 0.03439498692750931
298: accuracy:0.3404255319148936 loss: 0.0214575622

# N-new 0.6879432624113475 112轮

In [4]:
inputs = []
MAX_LENGTH = 53
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=1)
    a = np.log(a)
    if a.shape[0] < MAX_LENGTH:
        add = MAX_LENGTH - a.shape[0]
        adds = np.zeros(add)
        a = np.concatenate((a, adds))
    inputs.append(a)

In [5]:
use = np.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation

In [6]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))

In [7]:
b.shape,use.shape

((467, 53, 200), (467, 53))

In [8]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [9]:
t_s = []
t_e = []
t_y = []
for E,S,Y in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape

(torch.Size([141, 53]), torch.Size([141, 53, 200]), torch.Size([141]))

In [98]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=65 ,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,(h_n,c_n)=self.rnn(x.float())
        sen = sen.reshape((sen.shape[0],sen.shape[1],1))  # (BATCH_SIZE,53,1)
        output1 = torch.bmm(torch.transpose(output,1,2).float(),sen.float())  # (BATCH_SIZE,64,1)
        output1 = torch.sigmoid(output1)
        output1 = torch.softmax(output1,1)
        output1 = torch.bmm(output,output1) # (BATCH_SIZE,53,1)
        output = torch.cat((output1,output),2) # (BATCH_SIZE,53,65)
        output = self.maxpool(output)   # (BATCH_SIZE,1,65)
        output = output.reshape((output.shape[0],-1)) # (BATCH_SIZE,7)
        x1 = self.out(output.float())
        return x1

In [99]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
    drop_last=True,
)
net = RNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 1: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN(
  (rnn): LSTM(200, 64, batch_first=True)
  (out): Linear(in_features=65, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.3049645390070922 loss: 1.9433010816574097
2: accuracy:0.3333333333333333 loss: 1.8462063074111938
3: accuracy:0.4326241134751773 loss: 1.6321767568588257
4: accuracy:0.49645390070921985 loss: 1.5053471326828003
5: accuracy:0.5106382978723404 loss: 1.2786084413528442
6: accuracy:0.5390070921985816 loss: 1.175156831741333
7: accuracy:0.524822695035461 loss: 1.0732951164245605
8: accuracy:0.5531914893617021 loss: 0.9134953022003174
9: accuracy:0.5815602836879432 loss: 0.7435367107391357
10: accuracy:0.5815602836879432 loss: 0.6175999641418457
11: accuracy:0.5815602836879432 loss: 0.5391039252281189
12: accuracy:0.5815602836879432 loss: 0.4725273847579956
13: accuracy:0.5886524822695035 loss: 0.34439972043037415
14: accuracy:0.6028368794326241 loss: 0.24604764580726624
15:

137: accuracy:0.6666666666666666 loss: 0.0009977295994758606
138: accuracy:0.6666666666666666 loss: 0.00104542076587677
139: accuracy:0.6666666666666666 loss: 0.0009148344397544861
140: accuracy:0.6666666666666666 loss: 0.0008632875978946686
141: accuracy:0.6666666666666666 loss: 0.0010071024298667908
142: accuracy:0.6666666666666666 loss: 0.0009018294513225555
143: accuracy:0.6666666666666666 loss: 0.0010723061859607697
144: accuracy:0.6666666666666666 loss: 0.001063574105501175
145: accuracy:0.6666666666666666 loss: 0.0008732005953788757
146: accuracy:0.6666666666666666 loss: 0.0009153597056865692
147: accuracy:0.6666666666666666 loss: 0.0008699037134647369
148: accuracy:0.6666666666666666 loss: 0.0009642206132411957
149: accuracy:0.6666666666666666 loss: 0.0008136220276355743
150: accuracy:0.6595744680851063 loss: 0.0009157955646514893
151: accuracy:0.6595744680851063 loss: 0.0007381327450275421
152: accuracy:0.6595744680851063 loss: 0.0007743127644062042
153: accuracy:0.65957446808

# N-new-bi 0.6879432624113475 112轮

In [10]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class bi_RNN_new(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=True
        )
        self.out=torch.nn.Linear(in_features=129 ,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,(h_n,c_n)=self.rnn(x.float())
        sen = sen.reshape((sen.shape[0],sen.shape[1],1))  # (BATCH_SIZE,53,1)
        output1 = torch.bmm(torch.transpose(output,1,2).float(),sen.float())  # (BATCH_SIZE,64,1)
        output1 = torch.sigmoid(output1)
        output1 = torch.softmax(output1,1)
        output1 = torch.bmm(output,output1) # (BATCH_SIZE,53,1)
        output = torch.cat((output1,output),2) # (BATCH_SIZE,53,65)
        output = self.maxpool(output)   # (BATCH_SIZE,1,65)
        output = output.reshape((output.shape[0],-1)) # (BATCH_SIZE,7)
        x1 = self.out(output.float())
        return x1

In [11]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
    drop_last=True,
)
net = bi_RNN_new()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 1: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

bi_RNN_new(
  (rnn): LSTM(200, 64, batch_first=True, bidirectional=True)
  (out): Linear(in_features=129, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.2198581560283688 loss: 1.9409806728363037
2: accuracy:0.3829787234042553 loss: 1.7989925146102905
3: accuracy:0.41134751773049644 loss: 1.6326566934585571
4: accuracy:0.425531914893617 loss: 1.2530981302261353
5: accuracy:0.46808510638297873 loss: 1.2234959602355957
6: accuracy:0.574468085106383 loss: 1.0577131509780884
7: accuracy:0.6099290780141844 loss: 0.8930600881576538
8: accuracy:0.6028368794326241 loss: 0.6624197363853455
9: accuracy:0.6099290780141844 loss: 0.5008664131164551
10: accuracy:0.574468085106383 loss: 0.41010335087776184
11: accuracy:0.6453900709219859 loss: 0.34178754687309265
12: accuracy:0.6170212765957447 loss: 0.2322080135345459
13: accuracy:0.6312056737588653 loss: 0.15362492203712463
14: accuracy:0.6170212765957447

Traceback (most recent call last):
  File "/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
   

KeyboardInterrupt: 